## Problem Set 02
### AST 8110, Fall 2022
##### Tyler Barna

In [1]:
## Importing packages (some of these may not be useful)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import time
#bigstart = time.time()
#import numba

import sys
import os
import astropy
import astropy.units as u
import astropy.constants as const
import astropy.coordinates as coord
import random

from tqdm import tqdm
from scipy.stats import binned_statistic as binstat
from scipy import integrate

## Set Seed
random.seed(8110)
